In [2]:
# From Chris' visualise code

from extract_data import *

# Load the data from here using pickle (much faster)

cat_data = load_list('pickle_data', 'cat_data')
print('cat_data loaded')
uncat_data = load_list('pickle_data', 'uncat_data')
print('uncat_data loaded')
unlinked_data = load_list('pickle_data', 'unlinked_data')
print('unlinked_data loaded')

cat_data loaded
uncat_data loaded
unlinked_data loaded


In [15]:
from tslearn.clustering import TimeSeriesKMeans, KernelKMeans, KShape
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from math import sqrt

def prep_for_cluster_2(X, test, tilt, norm=False, stan=False):
    
    ##### X_TRAIN
    X_train = np.empty([72,len(X)])
    for i in range(len(X)):
        X_train[:,i] = X[i][1][tilt]
    
    X_train = np.transpose(X_train)
    
    # normalise the data
    if norm==True:
        for i in range(len(X_train)):
            X_train[i] = (X_train[i]-X_train[i].min())/(X_train[i].max()-X_train[i].min())
        
    # standardise the data
    if stan==True:
        scaler = StandardScaler()
        scaler = scaler.fit(X_train)
        X_train = scaler.transform(X_train)
    
    ##### X_TEST
    X_test = np.empty([72,len(test)])
    for i in range(len(test)):
        X_test[:,i] = test[i][1][tilt]
    
    X_test = np.transpose(X_test)
    
    # normalise the data
    if norm==True:
        for i in range(len(X_test)):
            X_test[i] = (X_test[i]-X_test[i].min())/(X_test[i].max()-X_test[i].min())
        
    # standardise the data
    if stan==True:
        scaler = StandardScaler()
        scaler = scaler.fit(X_test)
        X_test = scaler.transform(X_test)
    
    ##### Y_TEST
    
    y_test = []
    for i in range(len(test)):
        incident = cat_data[i]
        if 'Correct' in incident[3]:
            y_test.append(1)
        else:
            y_test.append(0)
       
    return X_train, X_test, y_test
    
X_train, X_test, y_test = prep_for_cluster_2(unlinked_data, cat_data, 'tiltx', norm=False, stan=False)

km = TimeSeriesKMeans(n_clusters=2, max_iter=10, metric="dtw", random_state=0).fit(X_train)
predictions = km.predict(X_test)
acc = accuracy_score(y_test, predictions)

print(predictions)
print("Accuracy:", acc)
print(confusion_matrix(y_test, predictions))

[1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 0 1 0 1 1 1 0 1 1 1 1 0 1 0 1 0 1 0 1 1 1
 1 0 1 1 0 0 0 1 0 1 1 1 0 1 1 0 0 0 1 1 0 0 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1
 1 0 0 0 0 0 1 1 0 0 0 0 1 1 0 1 1 1 1 1 1]
Accuracy: 0.47368421052631576
[[32 47]
 [ 3 13]]


c:\users\finnm\appdata\local\programs\python\python37\lib\site-packages\tslearn\utils\utils.py:89: UserWarning: 2-Dimensional data passed. Assuming these are 95 1-dimensional timeseries
  '{} 1-dimensional timeseries'.format(X.shape[0]))
